In [10]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas
import numpy as np
import requests
import argparse
import warnings

In [11]:
from p_reporting import p_reporting as rp
    
def argument_parser():
    parser = argparse.ArgumentParser(description='comandos para obtener información de tus parques favoritos')
    
    parser.add_argument("-l", "--listado", help="listado con todos los nombres de los parques de la Comunidad de Madrid" , type=str)
    parser.add_argument("-bs", "--bicimad_station", help="nombre de la estación de BiciMAD más cercana" , type=str)
    parser.add_argument("-bm", "--bicimad_station_meters", help="distancia a la estación de BiciMAD más cercana" , type=str)
    parser.add_argument("-ba", "--bicimad_adress", help="dirección de la estación de BiciMAD más cercana" , type=str)
    parser.add_argument("-bb", "--bicimad_bikes", help="número de bicis disponibles" , type=str)
    parser.add_argument("-bd", "--bicimad_dejar_bici", help="base disponible para dejar la bici" , type=str)
    parser.add_argument("-pd", "--place_description", help="descripción del parque" , type=str)
    parser.add_argument("-pb", "--place_barrio", help="barrio en el que se encuentra el parque" , type=str)
    parser.add_argument("-pt", "--place_transport", help="transporte público cercano al parque" , type=str)

    args = parser.parse_args()
    return args


def main(arguments):
    
    location_min_dataset = "datasets/final_df_min_distance_optimizated.csv"
    interaction_min_dataset = rp.import_min_dataset(location_min_dataset)
    
    if arguments.listado is None or arguments.bicimad_station is None or arguments.bicimad_adress is None or arguments.bicimad_bikes is None or arguments.bicimad_dejar_bici is None or arguments.place_description is None or arguments.place_barrio is None or arguments.place_transport is None:
        print("Hola, esta es una app informativa sobre los Parques Municipales de la excelentisima Comunidad de Madrid, presidida por nuestra dueña y señora AYUSO.\n\n Escribe: 'wikiparque.py -h' para saber todo lo que podemos ofrecerte")
        
    elif arguments.listado == "":
        print(location_min_dataset["Place of interest"].tolist())
        print("\n")
        print("Si quieres más información escribe uno de los comandos que te mostramos en 'wikiparque.py -h'")
           
    elif arguments.bicimad_station != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        bicimad_station2 = rp.bicimad_station(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(bicimad_station2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.bicimad_adress != "": #continuar con todos igual que el anterior
        input_parque = input("¿Cuál es tu parque favorito?: ")
        bicimad_adress2 = rp.bicimad_adress(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(bicimad_adress2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.bicimad_bikes != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        bicimad_bikes2 = rp.bicimad_bikes(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(bicimad_bikes2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.bicimad_dejar_bici != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        bicimad_dejar_bici2 = rp.bicimad_dejar_bici(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(bicimad_dejar_bici2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.place_description != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        place_description2 = rp.place_description(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(place_descriptions2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.place_barrio != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        place_barrio2 = rp.place_barrio(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(place_barrio2)
        else:
            print("Parque mal escrito... busca en Google cachondo")
            
    elif arguments.place_transport != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        place_transport2 = rp.place_transport(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(place_transport2)
        else:
            print("Parque mal escrito... busca en Google cachondo")

    elif arguments.bicimad_station_meters != "":
        input_parque = input("¿Cuál es tu parque favorito?: ")
        bicimad_station_meters2 = rp.bicimad_station_meters(input_parque,interaction_min_dataset)
        if input_parque in location_min_dataset["Place of interest"].tolist():
            print(bicimad_station_meters2)
        else:
            print("Parque mal escrito... busca en Google cachondo")            
            
    else:
        print("Comando mal escrito, escribe 'wikiparque.py -h' para volver a ver el conjunto de comandos")     
                  
# Pipeline execution

if __name__ == '__main__':
    main(argument_parser())

ImportError: cannot import name 'p_reporting' from 'p_reporting' (unknown location)

"""
def info_ready(input_parque,interaction_min_dataset):
    bicimad_station = rp.bicimad_station(input_parque,interaction_min_dataset)
    bicimad_adress = rp.bicimad_adress(input_parque,interaction_min_dataset)
    bicimad_bikes = rp.bicimad_bikes(input_parque,interaction_min_dataset)
    bicimad_dejar_bici = rp.bicimad_dejar_bici(input_parque,interaction_min_dataset)
    place_description = rp.place_description(input_parque,interaction_min_dataset)
    place_barrio = rp.place_barrio(input_parque,interaction_min_dataset)
    place_transport = rp.place_transport(input_parque,interaction_min_dataset)  
""" 

In [ ]:
bicimad_station_meters(input_parque,interaction_min_dataset)